In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from multi_layer_ltm import *

import time
print(time.ctime())
start = time.time()

input_path = r'D:\WorkingOn\Co_Trade_Link_MFA\Co_1998_2019\New_Trade_2020\\'

Sat Jul 29 15:50:48 2023


## Data

In [2]:
def agg_product_trade(layer):
    
    end_use_title = ['BEVs', 'BCEs', 'S&A', 'C&C', 'MAG', 'CAT', 'PI&E', 'OTH']
    if layer in end_use_title:
        return 'Product'
    else:
        return layer

    
df_Trade = pd.read_csv(input_path+'Co_1988_2020_Trade_New.csv')
df_Trade.Layer_From = df_Trade.Layer_From.apply(agg_product_trade)
df_Trade.Layer_To = df_Trade.Layer_To.apply(agg_product_trade)
df_Trade = df_Trade.groupby(['Year','Layer_From','Exporter','Layer_To','Importer']).agg(NetWeight = ('CoNetWeight', 'sum'))
df_Trade = df_Trade.reset_index()
df_Trade

,Year,Layer_From,Exporter,Layer_To,Importer,NetWeight
0,1988,Material,AUS,Material,KOR,0.000006
1,1988,Material,AUT,Material,CHE,0.001794
2,1988,Material,AUT,Material,DEU,0.057153
3,1988,Material,AUT,Material,GRC,0.000295
4,1988,Material,BEL,Material,AUS,5.040000
...,...,...,...,...,...,...
207723,2020,Waste,ZAF,Waste,NGA,0.000729
207724,2020,Waste,ZAF,Waste,NLD,0.438494
207725,2020,Waste,ZAF,Waste,SWZ,0.001561
207726,2020,Waste,ZMB,Waste,RUS,0.008676


In [3]:
df_Production_Total = pd.read_csv(input_path+'Production_Consumption_Total_1988_2020.csv')
df_Production_Total = df_Production_Total.groupby(['Year','Layer_From','Exporter','Layer_To','Importer']).agg(NetWeight = ('CoNetWeight', 'sum'))
df_Production_Total = df_Production_Total.reset_index()
df_Production_Total

,Year,Layer_From,Exporter,Layer_To,Importer,NetWeight
0,1988,Material,AGO,Product,AGO,1.335649
1,1988,Material,ARE,Product,ARE,0.000477
2,1988,Material,ARG,Product,ARG,0.010150
3,1988,Material,AUT,Product,AUT,4.542723
4,1988,Material,BEL,Product,BEL,39.837711
...,...,...,...,...,...,...
13894,2020,Waste,USA,Secondary,USA,1778.075405
13895,2020,Waste,VNM,Secondary,VNM,22.825144
13896,2020,Waste,ZAF,Secondary,ZAF,22.127693
13897,2020,Waste,ZMB,Secondary,ZMB,2.804712


In [4]:
df_Flow_without_sec = pd.concat([df_Production_Total, df_Trade])
df_Flow_without_sec.NetWeight = df_Flow_without_sec.NetWeight / 1000  # t to kt
df_Flow_without_sec

,Year,Layer_From,Exporter,Layer_To,Importer,NetWeight
0,1988,Material,AGO,Product,AGO,1.335649e-03
1,1988,Material,ARE,Product,ARE,4.766035e-07
2,1988,Material,ARG,Product,ARG,1.015022e-05
3,1988,Material,AUT,Product,AUT,4.542723e-03
4,1988,Material,BEL,Product,BEL,3.983771e-02
...,...,...,...,...,...,...
207723,2020,Waste,ZAF,Waste,NGA,7.285250e-07
207724,2020,Waste,ZAF,Waste,NLD,4.384940e-04
207725,2020,Waste,ZAF,Waste,SWZ,1.561125e-06
207726,2020,Waste,ZMB,Waste,RUS,8.676445e-06


In [5]:
def add_secondary_to_mining(df_flow):
    
    df_sec_production = df_flow[(df_flow.Layer_From=='Waste')&(df_flow.Layer_To=='Secondary')]
    df_sec_production.Layer_From = 'Secondary'
    df_sec_production.Layer_To = 'Material'
    
    df_flow = pd.concat([df_flow, df_sec_production])
    
    return df_flow


df_Flow_with_sec = add_secondary_to_mining(df_Flow_without_sec)
df_Flow_with_sec

,Year,Layer_From,Exporter,Layer_To,Importer,NetWeight
0,1988,Material,AGO,Product,AGO,1.335649e-03
1,1988,Material,ARE,Product,ARE,4.766035e-07
2,1988,Material,ARG,Product,ARG,1.015022e-05
3,1988,Material,AUT,Product,AUT,4.542723e-03
4,1988,Material,BEL,Product,BEL,3.983771e-02
...,...,...,...,...,...,...
13894,2020,Secondary,USA,Material,USA,1.778075e+00
13895,2020,Secondary,VNM,Material,VNM,2.282514e-02
13896,2020,Secondary,ZAF,Material,ZAF,2.212769e-02
13897,2020,Secondary,ZMB,Material,ZMB,2.804712e-03


## Simulation

In [6]:
Layer_Seq = ['Ore', 'Material', 'Product', 'Use', 'Waste', 'Secondary']

layer_country_nodes, adj_matrix = create_supra_adj_matrix(df_Flow_with_sec[df_Flow_with_sec.Year==2019], Layer_Seq)

thred = 0.3
shock_severity = 0.5
buffer_capacity = shock_severity * thred # bufer < shock
seed = [('Product','USA')]
seed

[('Product', 'USA')]

In [7]:
diffuse_all_rounds(layer_country_nodes, adj_matrix, shock_severity, buffer_capacity, seed)

[[('Product', 'USA')],
 [('Product', 'GIN'),
  ('Product', 'GUY'),
  ('Product', 'LCA'),
  ('Product', 'TKL'),
  ('Product', 'MEX'),
  ('Product', 'MSR'),
  ('Use', 'USA'),
  ('Product', 'BRB'),
  ('Product', 'GTM'),
  ('Material', 'USA'),
  ('Product', 'QAT'),
  ('Product', 'PRY'),
  ('Product', 'TTO'),
  ('Product', 'LVA'),
  ('Product', 'CAF'),
  ('Product', 'CMR'),
  ('Product', 'SHN'),
  ('Product', 'IRL'),
  ('Product', 'BLZ'),
  ('Product', 'AFG'),
  ('Product', 'FSM'),
  ('Product', 'ATG'),
  ('Product', 'ABW'),
  ('Product', 'LBR'),
  ('Product', 'AUS'),
  ('Product', 'CAN'),
  ('Product', 'AND'),
  ('Product', 'GRD'),
  ('Product', 'JAM'),
  ('Product', 'VCT'),
  ('Product', 'CCK'),
  ('Product', 'KNA'),
  ('Product', 'SUR'),
  ('Product', 'BHS')],
 [('Use', 'GUY'),
  ('Use', 'LCA'),
  ('Material', 'SUR'),
  ('Product', 'MDV'),
  ('Use', 'MEX'),
  ('Material', 'GIN'),
  ('Material', 'GBR'),
  ('Material', 'GUY'),
  ('Product', 'SXM'),
  ('Product', 'PLW'),
  ('Material', 'MEX